In [10]:
import pandas as pd
from ultralytics import YOLO

In [11]:
augmented_model_path = "./augmented.pt" # Local path where manually downloaded model from GCS is saved
test_results_path = "./test_results"  # Local folder for test results
test_data_yaml = "./test_data.yaml"  # YAML file for test data configuration

In [12]:
# Initialize augmented YOLO model
print("[INFO] Loading YOLO model...")
model = YOLO(augmented_model_path)

[INFO] Loading YOLO model...


# Initial Testing and Saving Results

In [13]:
model.val(
    data=test_data_yaml,  # Use the YAML file for test data
    save_json=True,       # Save results in JSON format
    save_conf=True,       # Save confidence scores
    project=test_results_path, # Save results in the specified folder
    name="augmented_test_predictions",
    plots=True            # Enable plotting for visualization
    )

# Display validation metrics from YOLO output
print("\nValidation completed. Check the metrics above and JSON file for detailed results.")

Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:13


                   all        163        878      0.759      0.838      0.864      0.729
                     V        162        694      0.815      0.867      0.912      0.733
                     C         79        100      0.582      0.767      0.747      0.644
                     S         64         84      0.881       0.88      0.933       0.81
Speed: 1.2ms preprocess, 55.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving test_results\augmented_test_predictions\predictions.json...
Results saved to test_results\augmented_test_predictions

Validation completed. Check the metrics above and JSON file for detailed results.


# Saving Results to csv files

In [4]:
def evaluate_and_save_metrics(model, test_data_yaml, conf_values, iou_values, output_dir):
    # Initialize lists for metrics
    metrics_all = []
    metrics_v = []
    metrics_c = []
    metrics_s = []

    # Iterate over all combinations of confidence and IoU thresholds
    for conf in conf_values:
        for iou in iou_values:
            print(f"Evaluating: conf = {conf}, IoU = {iou}")
            
            # Run model evaluation
            metrics = model.val(data=test_data_yaml, conf=conf, iou=iou, plots=False)

            # Access overall mean results
            mean_results = metrics.mean_results()  # Precision, recall, mAP@50, mAP@50-95
            metrics_all.append({
                "iou": iou,
                "conf": conf,
                "p": mean_results[0],  # Precision
                "r": mean_results[1],  # Recall
                "map50": mean_results[2],  # mAP@50
                "map50-95": mean_results[3]  # mAP@50-95
            })

            # Access per-class metrics
            for class_idx, metrics_list in zip(
                [0, 1, 2], [metrics_v, metrics_c, metrics_s]
            ):
                class_results = metrics.class_result(class_idx)  # Get metrics for a specific class
                metrics_list.append({
                    "iou": iou,
                    "conf": conf,
                    "p": class_results[0],  # Precision for the class
                    "r": class_results[1],  # Recall for the class
                    "map50": class_results[2],  # mAP@50 for the class
                    "map50-95": class_results[3]  # mAP@50-95 for the class
                })

    # Create DataFrames
    df_all = pd.DataFrame(metrics_all)
    df_v = pd.DataFrame(metrics_v)
    df_c = pd.DataFrame(metrics_c)
    df_s = pd.DataFrame(metrics_s)

    # Save DataFrames to CSV files
    df_all.to_csv(f"{output_dir}/metrics_all.csv", index=False)
    df_v.to_csv(f"{output_dir}/metrics_v.csv", index=False)
    df_c.to_csv(f"{output_dir}/metrics_c.csv", index=False)
    df_s.to_csv(f"{output_dir}/metrics_s.csv", index=False)

    print("All metrics saved successfully.")

In [5]:
conf_values = [0.001, 0.1, 0.3, 0.5]
iou_values = [0.3, 0.5, 0.7, 0.9]
output_dir = "./augmented_results"

evaluate_and_save_metrics(model, test_data_yaml, conf_values, iou_values, output_dir)

Evaluating: conf = 0.001, IoU = 0.3
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:21

                   all        163        878       0.75      0.844      0.865      0.729
                     V        162        694      0.804      0.871      0.914      0.736
                     C         79        100      0.585       0.78      0.749      0.643
                     S         64         84      0.861      0.883      0.933      0.809
Speed: 1.4ms preprocess, 83.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Evaluating: conf = 0.001, IoU = 0.5
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:20

                   all        163        878      0.761      0.839      0.865      0.729
                     V        162        694      0.817      0.867      0.914      0.735
                     C         79        100      0.587       0.77      0.748      0.643
                     S         64         84       0.88      0.881      0.932      0.808
Speed: 1.3ms preprocess, 77.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Evaluating: conf = 0.001, IoU = 0.7
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


...

                   all        163        878      0.843      0.728      0.807      0.706
                     V        162        694       0.88      0.755       0.85      0.717
                     C         79        100      0.707       0.63      0.684      0.598
                     S         64         84      0.944      0.798      0.887      0.803
Speed: 1.3ms preprocess, 75.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Evaluating: conf = 0.5, IoU = 0.7
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:20

                   all        163        878      0.843      0.728      0.807      0.706
                     V        162        694       0.88      0.755       0.85      0.717
                     C         79        100      0.707       0.63      0.684      0.598
                     S         64         84      0.944      0.798      0.887      0.803
Speed: 1.2ms preprocess, 75.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Evaluating: conf = 0.5, IoU = 0.9
Ultralytics 8.3.54  Python-3.12.1 torch-2.5.1+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)



val: Scanning C:\Users\binmh\AI group Project\test\labels.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|█████████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:20

                   all        163        878      0.825       0.72        0.8      0.701
                     V        162        694       0.83      0.743      0.837      0.707
                     C         79        100      0.729      0.619      0.682      0.596
                     S         64         84      0.918      0.798      0.883      0.799
Speed: 1.3ms preprocess, 76.4ms inference, 0.0ms loss, 0.9ms postprocess per image
All metrics saved successfully.
